In [1]:
import json 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from funcs import fit_ar_garch, reconstruct_residuals, get_imbalance_price, fit_sample_copula
from copulas.univariate import GaussianKDE, BetaUnivariate, GaussianUnivariate, GammaUnivariate, LogLaplace, StudentTUnivariate
from fetch_da_prices import load_and_process_boiler_data, get_day_ahead_prices
%load_ext autoreload
%autoreload 2

In [2]:
start = pd.Timestamp('20250304', tz='UTC')
end = pd.Timestamp('20250504', tz='UTC')

In [3]:
capacity_up_hat = pd.read_parquet('/home/alqua/git/afrr-price-forecast-engine/data/results/afrr_price_forecast_UP.parquet')
da_prices_hat = load_and_process_boiler_data("/home/alqua/data/boiler_data/daily_data")

da_prices = get_day_ahead_prices(start, end, price_area="DK_1")
da_prices_hat = da_prices_hat.loc[start:end]
pic_prices = get_imbalance_price(start_date=start, end_date=end, price_area="DK1")
capacity_up_hat.index = capacity_up_hat.index.tz_localize('UTC')
pic_prices.index = pic_prices.index.rename('time_utc')

pic_prices.index= pd.to_datetime(pic_prices.index, utc=True)
capacity_up_hat = capacity_up_hat.loc[start:end]

In [4]:
da_prices_df = da_prices_hat.join(da_prices, on="time_utc")
da_prices_df["da_price"] = da_prices_df["da_price"].ffill()
da_prices_df["da_resid"] = da_prices_df["da_preds"] - da_prices_df["da_price"]


In [5]:
da_prices_df["da_resid"] = da_prices_df["da_resid"].diff(1)

In [6]:
data = pic_prices.join(capacity_up_hat, on='time_utc' )
data = pic_prices.join(capacity_up_hat, on='time_utc' )
data['pic_up_resid'] =  data['aFRRVWAUpEUR'] - data['SpotPriceEUR']



In [7]:
data = data.join(da_prices_df, on='time_utc', how='inner')

In [8]:
data = data.ffill()

In [9]:
ar_params_cap_up, garch_params_cap_up, std_res_cap_up, cond_vol_up = fit_ar_garch(data['cap_up_resid'], ar_order=1, garch_p=1, garch_q=1)
ar_params_pic_up, garch_params_pic_up, std_res_cap_pic_up, cond_vol_pic_up = fit_ar_garch(data['pic_up_resid'], ar_order=1, garch_p=1, garch_q=1)
ar_params_da, garch_params_da, std_res_da, cond_vol_da = fit_ar_garch(data['da_resid'], ar_order=1, garch_p=1, garch_q=1)

data['cap_up_resid_std'] = std_res_cap_up
data['pic_up_resid_std'] = std_res_cap_pic_up
data['da_resid_std'] = std_res_cap_pic_up

/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/home/alqua/anaconda3/envs/opt_env/lib/python3.12/site-packages/statsmodels/tsa/

In [10]:
from copulas.multivariate import GaussianMultivariate
import numpy as np

dist = GaussianMultivariate(distribution={"cap_up_resid": GaussianKDE, 
                                         "pic_up_resid": GaussianKDE, 
                                         "da_resid": GaussianKDE})

dist.fit(data[["cap_up_resid", "pic_up_resid", "da_resid"]])

samples = dist.sample(10000)

In [11]:
def sample_quantile(samples, dist, target_quantile):
    cdf_values = samples.apply(dist.cumulative_distribution, axis=1)
    idx_closest = (cdf_values - target_quantile).abs().idxmin()
    quantile_sample = samples.loc[idx_closest]
    
    return quantile_sample
    

In [14]:
joint_quantile_q90 = sample_quantile(samples, dist, 0.9)
joint_quantile_q50 = sample_quantile(samples, dist, 0.5)
joint_quantile_q10 = sample_quantile(samples, dist, 0.1)

In [15]:
data['cap_up_resid_q90'] = joint_quantile_q90['cap_up_resid']
data['cap_up_resid_q50'] = joint_quantile_q50['cap_up_resid']
data['cap_up_resid_q10'] = joint_quantile_q10['cap_up_resid']

In [28]:
joint_quantile_q90['cap_up_resid']

np.float64(43.35269176110975)

In [ ]:
data['opt_afrr_up_cap_price_q90'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q90']
data['opt_afrr_up_cap_price_q50'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q50']
data['opt_afrr_up_cap_price_q10'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q10']


In [55]:
data.columns

Index(['BalancingDemand', 'SpotPriceEUR', 'DominatingDirection', 'aFRRUpMW',
       'aFRRVWAUpEUR', 'aFRRDownMW', 'aFRRVWADownEUR', 'lr_afrr_up_cap_price',
       'xgb_afrr_up_cap_price', 'afrr_up_cap_price', 'opt_afrr_up_cap_price',
       'cap_up_resid', 'pic_up_resid', 'heat_load_forecast', 'da_preds',
       'da_price', 'da_resid', 'cap_up_resid_std', 'pic_up_resid_std',
       'da_resid_std', 'cap_up_resid_q90', 'cap_up_resid_q50',
       'cap_up_resid_q10', 'opt_afrr_up_cap_price_q90',
       'opt_afrr_up_cap_price_q50', 'opt_afrr_up_cap_price_q10'],
      dtype='object')

In [66]:
da_q90_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.1)

da_q90_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.1)

da_q90_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.1)

data['da_q90_cond_cap_q10'] = data['da_preds'] + da_q90_cond_cap_q10
data['da_q90_cond_cap_q50'] = data['da_preds'] + da_q90_cond_cap_q50
data['da_q90_cond_cap_q90'] = data['da_preds'] + da_q90_cond_cap_q90
data['da_q50_cond_cap_q10'] = data['da_preds'] + da_q50_cond_cap_q10
data['da_q50_cond_cap_q50'] = data['da_preds'] + da_q50_cond_cap_q50
data['da_q50_cond_cap_q90'] = data['da_preds'] + da_q50_cond_cap_q90
data['da_q10_cond_cap_q10'] = data['da_preds'] + da_q10_cond_cap_q10
data['da_q10_cond_cap_q50'] = data['da_preds'] + da_q10_cond_cap_q50
data['da_q10_cond_cap_q90'] = data['da_preds'] + da_q10_cond_cap_q90


In [71]:
# Brach 1 Picasso
pic_q90_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 2 Picasso
pic_q90_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 3 Picasso
pic_q90_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)
# Brach 4 Picasso
pic_q90_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 5 Picasso
pic_q90_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 6 Picasso
pic_q90_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)
# Brach 7 Picasso
pic_q90_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 8 Picasso
pic_q90_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 9 Picasso
pic_q90_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)


In [72]:
# Assuming the quantiles are already calculated as per your branches

# Create new columns with specific names based on the quantile combinations
data['pic_q90_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q90
data['pic_q50_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q90
data['pic_q10_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q90

data['pic_q90_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q90
data['pic_q50_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q90
data['pic_q10_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q90

data['pic_q90_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q90
data['pic_q50_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q90
data['pic_q10_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q90

data['pic_q90_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q50
data['pic_q50_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q50
data['pic_q10_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q50

data['pic_q90_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q50
data['pic_q50_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q50
data['pic_q10_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q50

data['pic_q90_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q50
data['pic_q50_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q50
data['pic_q10_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q50

data['pic_q90_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q10
data['pic_q50_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q10
data['pic_q10_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q10

data['pic_q90_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q10
data['pic_q50_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q10
data['pic_q10_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q10

data['pic_q90_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q10
data['pic_q50_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q10
data['pic_q10_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q10

In [ ]:
"""from scipy.stats import gaussian_kde

def calculate_probabilities(group, forecast_col, historical_col, col_name):
    kde = gaussian_kde(group[historical_col])
    x = np.linspace(group[historical_col].min(), group[historical_col].max(), 1000)
    density = kde(x)

    scenario_values = {
        f'{col_name}_q90': group[f'{forecast_col}_q90'].mean(),
        f'{col_name}_q50': group[f'{forecast_col}_q50'].mean(),
        f'{col_name}_q10': group[f'{forecast_col}_q10'].mean()
    }

    probabilities = {}
    for scenario, value in scenario_values.items():
        cdf_value = np.trapezoid(density[x <= value], x[x <= value])  # Area under the curve
        probabilities[scenario] = cdf_value

    total_probability = sum(probabilities.values())
    normalized_probabilities = {k: v / total_probability for k, v in probabilities.items()}

    return normalized_probabilities"""

In [ ]:
'''
forecast_column = 'da_preds'  
historical_column = 'da_price'  

daily_probabilities_da = data.groupby(data.index.date).apply(calculate_probabilities, 
                                                         forecast_col=forecast_column, 
                                                         historical_col=historical_column, 
                                                         col_name='da_price_probs')

daily_probabilities_da_df = pd.DataFrame(list(daily_probabilities_da)).set_index(pd.to_datetime(daily_probabilities_da.index))
daily_probabilities_da_df.index = daily_probabilities_da_df.index.normalize()
#daily_probabilities_da_df'''

In [ ]:
'''
daily_probabilities_afrr = data.groupby(data.index.date).apply(calculate_probabilities, 
                                                         forecast_col='opt_afrr_up_cap_price', 
                                                         historical_col='afrr_up_cap_price', 
                                                         col_name='afrr_up_prices_probs')

daily_probabilities_afrr_df = pd.DataFrame(list(daily_probabilities_afrr)).set_index(pd.to_datetime(daily_probabilities_afrr.index))
daily_probabilities_afrr_df.index = daily_probabilities_afrr_df.index.normalize()
daily_probabilities_afrr_df'''

,afrr_up_prices_probs_q90,afrr_up_prices_probs_q50,afrr_up_prices_probs_q10
2025-03-04,0.333333,0.333333,0.333333
2025-03-05,0.336396,0.336396,0.327209
2025-03-06,0.428487,0.342959,0.228554
2025-03-07,0.594322,0.281786,0.123892
2025-03-08,0.511427,0.322069,0.166504
...,...,...,...
2025-04-29,0.588676,0.275092,0.136231
2025-04-30,0.482744,0.329289,0.187966
2025-05-01,0.439078,0.363415,0.197507
2025-05-02,0.464736,0.341914,0.193350


In [ ]:
'''daily_probabilities_pic = data.groupby(data.index.date).apply(calculate_probabilities, 
                                                         forecast_col='da_preds', 
                                                         historical_col='aFRRVWAUpEUR', 
                                                         col_name='picasso_price_probs')

daily_probabilities_pic_df = pd.DataFrame(list(daily_probabilities_pic)).set_index(pd.to_datetime(daily_probabilities_pic.index))
daily_probabilities_pic_df.index = daily_probabilities_pic_df.index.normalize()
#daily_probabilities_pic_df'''

In [26]:
import pandas as pd
import numpy as np

# Assuming daily_probabilities_da_df, daily_probabilities_afrr_df, and daily_probabilities_pic_df
# have already been created and have quantile columns.

# Define the quantile suffixes
quantile_suffixes = ['_q90', '_q50', '_q10']

# Initialize an empty dictionary to store the scenarios
scenarios_dict = {}
cnt = 0
# Loop through the quantile combinations
for q1 in range(len(quantile_suffixes)):
    for q2 in range(len(quantile_suffixes)):
        for q3 in range(len(quantile_suffixes)):
            # Create a key for the dictionary based on the current combination of quantiles
            key = (q1, q2, q3)
            cnt += 1
            # Construct the column names for the current scenario
            da_price_prob_col = f'da_price_probs{quantile_suffixes[q1]}'
            afrr_price_prob_col = f'afrr_up_prices_probs{quantile_suffixes[q2]}'
            picasso_price_prob_col = f'picasso_price_probs{quantile_suffixes[q3]}'

            # Loop through each row in the dataframes and store the last row's data
            for index in range(len(daily_probabilities_da_df)):
                # Get the probabilities for the current row
                da_price_prob = daily_probabilities_da_df.iloc[index][da_price_prob_col].round(2)
                afrr_price_prob = daily_probabilities_afrr_df.iloc[index][afrr_price_prob_col].round(2)
                picasso_price_prob = daily_probabilities_pic_df.iloc[index][picasso_price_prob_col].round(2)

                # Calculate total probability
                tot_prob = (da_price_prob * afrr_price_prob * picasso_price_prob).round(2)

                # Store the data in the dictionary with np.float64 and include quantile suffixes
                scenarios_dict[key] = {
                    'scenario_id': cnt,
                    f'da_price_probs{quantile_suffixes[q1]}': np.float64(da_price_prob),
                    f'afrr_up_prices_probs{quantile_suffixes[q2]}': np.float64(afrr_price_prob),
                    f'picasso_price_probs{quantile_suffixes[q3]}': np.float64(picasso_price_prob),
                    'tot_prob': np.float64(tot_prob)
                }

# Print the resulting dictionary
print(scenarios_dict)

{(0, 0, 0): {'scenario_id': 1, 'da_price_probs_q90': np.float64(0.49), 'afrr_up_prices_probs_q90': np.float64(0.53), 'picasso_price_probs_q90': np.float64(0.45), 'tot_prob': np.float64(0.12)}, (0, 0, 1): {'scenario_id': 2, 'da_price_probs_q90': np.float64(0.49), 'afrr_up_prices_probs_q90': np.float64(0.53), 'picasso_price_probs_q50': np.float64(0.29), 'tot_prob': np.float64(0.08)}, (0, 0, 2): {'scenario_id': 3, 'da_price_probs_q90': np.float64(0.49), 'afrr_up_prices_probs_q90': np.float64(0.53), 'picasso_price_probs_q10': np.float64(0.27), 'tot_prob': np.float64(0.07)}, (0, 1, 0): {'scenario_id': 4, 'da_price_probs_q90': np.float64(0.49), 'afrr_up_prices_probs_q50': np.float64(0.34), 'picasso_price_probs_q90': np.float64(0.45), 'tot_prob': np.float64(0.07)}, (0, 1, 1): {'scenario_id': 5, 'da_price_probs_q90': np.float64(0.49), 'afrr_up_prices_probs_q50': np.float64(0.34), 'picasso_price_probs_q50': np.float64(0.29), 'tot_prob': np.float64(0.05)}, (0, 1, 2): {'scenario_id': 6, 'da_price

In [ ]:
COLS = ['BalancingDemand', 'DominatingDirection', 'heat_load_forecast',
        
 'aFRRUpMW', 'aFRRVWAUpEUR', 'aFRRDownMW', 'aFRRVWADownEUR',
'opt_afrr_up_cap_price','cap_up_resid', 'opt_afrr_up_cap_price_q90', 'opt_afrr_up_cap_price_q50', 'opt_afrr_up_cap_price_q10',
'da_price', 'da_preds', 'da_resid', 'da_preds_q90', 'da_preds_q50', 'da_preds_q10',
'pic_up_resid', 'aFRRVWAUpEUR_90', 'aFRRVWAUpEUR_50',  'aFRRVWAUpEUR_10']

In [37]:
data[COLS].columns

Index(['BalancingDemand', 'SpotPriceEUR', 'DominatingDirection',
       'heat_load_forecast', 'aFRRUpMW', 'aFRRVWAUpEUR', 'aFRRDownMW',
       'aFRRVWADownEUR', 'opt_afrr_up_cap_price', 'cap_up_resid',
       'opt_afrr_up_cap_price_q90', 'opt_afrr_up_cap_price_q50',
       'opt_afrr_up_cap_price_q10', 'da_price', 'da_preds', 'da_resid',
       'da_preds_q90', 'da_preds_q50', 'da_preds_q10', 'pic_up_resid',
       'aFRRVWAUpEUR_90', 'aFRRVWAUpEUR_50', 'aFRRVWAUpEUR_10'],
      dtype='object')

In [38]:


rename_dict = {
                'BalancingDemand': 'BalancingDemand', #balancing demand 
                'DominatingDirection': 'DominatingDirection', #direction of the balancing demand 
                'heat_load_forecast': 'HeatLoadForecast', #heat schedule forecast
                
                'aFRRVWAUpEUR': 'PiCUpEUR', #Picasso imbalance price weighted by the volume 
                'aFRRUpMW': 'aFRRUpMW', #Capacity Prices MW
                'opt_afrr_up_cap_price': 'aFRRUpCapHat', #aFRRUpCap price forecast
                'opt_afrr_up_cap_price_q90': 'aFRRUpCapHatQ90', #aFRRUpCap price forecast q90
                'opt_afrr_up_cap_price_q50': 'aFRRUpCapHatQ50', #aFRRUpCap price forecast q50
                'opt_afrr_up_cap_price_q10': 'aFRRUpCapHatQ10', #aFRRUpCap price forecast q10
                'da_price': 'DayAhead', #day ahead real price, 
                'da_preds': 'DayAheadHat', #day ahead price forecast
                'da_preds_q90': 'DayAheadHatQ90', #day ahead price forecast q90
                'da_preds_q50': 'DayAheadHatQ50', #day ahead price forecast q50
                'da_preds_q10': 'DayAheadHatQ10', #day ahead price forecast q10
                'aFRRVWAUpEUR_90': 'PiCUpEURQ90', #Picasso price real residuals q90
                'aFRRVWAUpEUR_50': 'PiCUpEURQ50', 
                'aFRRVWAUpEUR_10': 'PiCUpEURQ10', #Picasso price real residuals q10
            }

In [41]:
data = data[COLS].rename(columns=rename_dict)
data.to_parquet('../data/results/data_for_optimization.parquet')

In [43]:
def convert_to_serializable(data):
    new_data = {}
    for key, value in data.items():
        new_key = str(key)  # Convert tuple to string
        new_value = {k: float(v) if isinstance(v, np.float64) else v for k, v in value.items()}
        new_data[new_key] = new_value
    return new_data

# Convert the data
serializable_data = convert_to_serializable(scenarios_dict)

In [51]:
with open('../data/results/scenarios_dict.json', 'w', encoding='utf-8') as json_file:
    json.dump(serializable_data, json_file, indent=4, ensure_ascii=False)